<a href="https://colab.research.google.com/github/andrew-veriga/Tensorflow-labs/blob/master/IMDB_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0

tf.config.run_functions_eagerly(True)


# !pip install -q tensorflow-datasets

2.4.0


Если вы используете Google Colab, то у вас уже есть установленные наборы данных TensorFlow. Если у вас их нет, они легко устанавливаются с помощью команды import.
Здесь мы используем датасет отзывов с imdb, команда tfds.load загрузит данные и их метаданные.

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Данные разделены на 25 000 примеров обучения и 25 000 примеров тестирования. В каждом из них содержатся предложения и метки в виде тензоров. 
В циклах создаются массивы numpy

In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


Посмотрите, как выглядит обзор

In [ ]:
print(training_sentences[2])
print(training_labels[2])

Далее мы токенизируем наши предложения.
Обратите внимание, что word_index - это слова, которые получены из учебного набора, поэтому вы будьте готовы встретить намного больше токенов слов во время теста.

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

Пришло время определить нашу нейронную сеть. Это должно выглядеть очень знакомым, за исключением, возможно, строки Embedding. Это ключ к анализу эмоций в текстах в TensorFlow, и именно здесь происходит волшебство.

Результатом Embedding будет двумерный массив с первой размерностью - длиной предложения и второй - размером вектора встраивания embedding_dim=16.

Затем  мы должны "расплющить" его так же, как мы это делали с изображениями.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    #в качестве альтернативы flatten можно использовать GlobalAveragePooling1D который берет среднее значение по каждому вектору, чтобы сделать его одномерным:
    #tf.keras.layers.GlobalAveragePooling1D(),
    # Он проще и немного быстрее.
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Результаты на наборе обучения дадут нам точность примерно 1,00, и на валидации - примерно 0,82. Скорее всего, здесь оверфиттинг. Мы рассмотрим некоторые стратегии, чтобы избежать этого позже
Теперь нам нужно визуализировать embedding. Мы начнем с получения результатов слоя Embedding, который в нашей модели -  нулевой слой. Мы можем получить веса и вывести их размерность следующим образом:

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

Мы можем видеть, что это массив 10000х16, у нас есть 10 000 слов в нашем корпусе, и мы работаем в 16-мерном массиве. Чтобы визуализовать его, нам нужна вспомогательная функция, чтобы вернуть наш word_index обратно в слова. Функция `reverse_word_index` была определена выше.
Запишем векторы и их метаданные в файлы.  TensorFlow Projector читает этот тип файла и использует его для построения векторов в трехмерном пространстве, чтобы мы могли их визуализировать.
В файл векторов мы просто записываем значение каждого из элементов в массиве встраиваний, т.е. значение в каждом измерении вектора для этого слова. Для массива метаданных мы просто записываем соответствующие векторам слова.

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

Выгрузить эти файлы можно просто из навигатора, которое находится справа в меню "files", или при помощи кода ниже.
Чтобы визуализировать результаты, перейдите в TensorFlow Embedded Projector [https://projector.tensorflow.org] и нажмите там кнопку «Load data» слева.
Включите флажок 'sphereize data' в левом верхнем углу, и вы увидите двоичную кластеризацию данных. Экспериментируйте с поиском слов, или нажимайте на синие точки на диаграмме, представляющие слова.

In [ ]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)